# Solution algorythm step by step (with thought process and comments):
### 1) Load and transform the data for further calculations

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_dict = pd.read_excel('Slot Math Test.xlsx', sheet_name=None)
df_dict

{'Summary':                    Fomat: 5-reel, 3-row Unnamed: 1 Unnamed: 2
 0                           Paylines: 5        NaN        NaN
 1    Maximum win on a single bet line:        1000    x wager
 2   Features: Wild, Scatter, Free Spins        NaN        NaN
 3                                   NaN        NaN        NaN
 4                                   RTP        ???        NaN
 5                      Volatility Index        ???        NaN
 6                                 Hit %        NaN        NaN
 7                              Per Line        ???        NaN
 8                 Per Spin (5 lines on)        ???        NaN
 9                     Bonus Game Chance        ???        NaN
 10  Total Hit (any line or scatter win)        ???        NaN,
 'Rules and Features':                                            Game Rules
 0   Payline wins are multiplied by the amount stak...
 1     Coinciding wins on different paylines are added
 2   Bet lines win if the winning symbols are

In [3]:
df_dict.keys()

dict_keys(['Summary', 'Rules and Features', 'Symbols', 'Pay Table', 'Reels'])

In [4]:
df_dict['Reels']

,Unnamed: 0,Explicit,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Position,Reel 1,Reel 2,Reel 3,Reel 4,Reel 5
1,1.0,S,W,J,K,J
2,2.0,Q,K,Q,J,Q
3,3.0,J,J,S,Q,A
4,4.0,Q,K,J,K,J
5,5.0,A,J,K,A,Q
6,6.0,K,A,J,J,S
7,7.0,Q,J,S,K,J
8,8.0,S,K,J,J,K
9,9.0,Q,J,K,K,J


In [5]:
df_dict['Reels'].drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
df_dict['Reels']

,Explicit,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Reel 1,Reel 2,Reel 3,Reel 4,Reel 5
1,S,W,J,K,J
2,Q,K,Q,J,Q
3,J,J,S,Q,A
4,Q,K,J,K,J
5,A,J,K,A,Q
6,K,A,J,J,S
7,Q,J,S,K,J
8,S,K,J,J,K
9,Q,J,K,K,J


In [7]:
df_dict['Reels'].columns = df_dict['Reels'].iloc[0]
df_dict['Reels'] = df_dict['Reels'].drop(0)

In [8]:
df_dict['Reels']

,Reel 1,Reel 2,Reel 3,Reel 4,Reel 5
1,S,W,J,K,J
2,Q,K,Q,J,Q
3,J,J,S,Q,A
4,Q,K,J,K,J
5,A,J,K,A,Q
6,K,A,J,J,S
7,Q,J,S,K,J
8,S,K,J,J,K
9,Q,J,K,K,J
10,A,A,Q,Q,Q


In [9]:
df_dict['Reels']['Reel 1']

1     S
2     Q
3     J
4     Q
5     A
6     K
7     Q
8     S
9     Q
10    A
11    Q
12    J
13    A
14    S
15    Q
16    J
17    A
18    K
19    Q
20    J
21    Q
Name: Reel 1, dtype: object

In [10]:
df_reels = df_dict['Reels'].copy()
df_reels[:1]

,Reel 1,Reel 2,Reel 3,Reel 4,Reel 5
1,S,W,J,K,J


### 2) Calculating every type of hit %
- First I intended to include the bonus game chance into the other hit %, but there are no "per bet" hit %, so it's sufficient to just calculate the raw hits that were stated in the tables. We might need to use the bonus game data later to calculate the RTP
- As the rules state, only the highest paying combo per payline counts, so this will influence our calculations. We will go from the highest paying combo down

In [11]:
# this format will represent probabilities of symbols for each reel

symbols = ['W', 'A', 'K', 'Q', 'J', 'S']
data = {reel : range(6) for reel in df_reels.columns}

df_example = pd.DataFrame(data, index=symbols)
df_example

,Reel 1,Reel 2,Reel 3,Reel 4,Reel 5
W,0,0,0,0,0
A,1,1,1,1,1
K,2,2,2,2,2
Q,3,3,3,3,3
J,4,4,4,4,4
S,5,5,5,5,5


In [12]:
np.count_nonzero(df_reels['Reel 1'] == 'A')

4

In [13]:
len(df_reels['Reel 1'])

21

In [14]:
# probability building blocks for better understanding of my thought process later:

prob_A_reel1 = np.count_nonzero(df_reels['Reel 1'] == 'A')/len(df_reels['Reel 1'])
prob_A_reel1

0.19047619047619047

In [15]:
prob_not_A_reel1 = 1 - prob_A_reel1
prob_not_A_reel1

0.8095238095238095

In [16]:
symbols = ['W', 'A', 'K', 'Q', 'J', 'S']

data = {reel : [
    np.count_nonzero(df_reels[reel] == s)/len(df_reels[reel])
    for s in symbols
] for reel in df_reels.columns}

df_probs = pd.DataFrame(data, index=symbols)
df_probs

,Reel 1,Reel 2,Reel 3,Reel 4,Reel 5
W,0.000000,0.047619,0.047619,0.047619,0.047619
A,0.190476,0.095238,0.095238,0.190476,0.095238
K,0.095238,0.285714,0.142857,0.333333,0.142857
Q,0.380952,0.142857,0.238095,0.190476,0.285714
J,0.190476,0.428571,0.333333,0.238095,0.285714
S,0.142857,0.000000,0.142857,0.000000,0.142857


In [17]:
df_probs['Reel 1'].loc['A']

0.19047619047619047

In [18]:
# now we calculate the symbol combo chances using the above info:

a5_per_payline = (df_probs['Reel 1'].loc['A'] + df_probs['Reel 1'].loc['W']
                 ) * (df_probs['Reel 2'].loc['A'] + df_probs['Reel 2'].loc['W']
                 ) * (df_probs['Reel 3'].loc['A'] + df_probs['Reel 3'].loc['W']
                 ) * (df_probs['Reel 4'].loc['A'] + df_probs['Reel 4'].loc['W']
                 ) * (df_probs['Reel 5'].loc['A'] + df_probs['Reel 5'].loc['W'])

a5_per_payline

0.00013222004059155242

In [19]:
# can be automated with a function. let's try, including logic from the rules on payout:

def prob_per_payline(length, s):
    prob = 1
    for reel in range(length):
        prob = prob * (
        df_probs[df_probs.columns[reel]].loc[s] + df_probs[df_probs.columns[reel]].loc['W'])
    for reel_2 in range(reel, 4):
        prob = prob * (
        1 - (
        df_probs[
            df_probs.columns[reel_2]].loc[s] + df_probs[df_probs.columns[reel_2]].loc['W']))
    return prob

prob_per_payline(5, 'A')

0.00013222004059155242

In [20]:
# seems to work. let's now do a table for combo probs and move to calculating those:

combo_symbols = ['A', 'K', 'Q', 'J']
data = {length : 0 for length in [3, 4, 5]}

df_combo_probs_example = pd.DataFrame(data, index=combo_symbols)
df_combo_probs_example

,3,4,5
A,0,0,0
K,0,0,0
Q,0,0,0
J,0,0,0


In [21]:
# let's see if we can automate the combo chance calculations:

df_combo_probs_example[5].loc['A'] = prob_per_payline(5, 'A')
df_combo_probs_example[5].loc['K'] = prob_per_payline(5, 'K')
df_combo_probs_example

,3,4,5
A,0,0,0.000132
K,0,0,0.000439
Q,0,0,0.000000
J,0,0,0.000000


In [22]:
# seems automatable. let's try:

combo_symbols = ['A', 'K', 'Q', 'J']

data = {length : [
    prob_per_payline(length, s) for s in combo_symbols
] for length in [3, 4, 5]}

df_combo_probs = pd.DataFrame(data, index=combo_symbols)
df_combo_probs

,3,4,5
A,0.002539,0.000705,0.000132
K,0.003030,0.001426,0.000439
Q,0.011283,0.003761,0.001645
J,0.015279,0.007052,0.003291


In [23]:
# with this data we can calculate the hit percents

hit_percent_per_payline = df_combo_probs.sum().sum() * 100
hit_percent_per_payline

5.0581511084079445

In [24]:
# hit % per spin is not influenced by scatter symbols, because scatter symbols activate an
# additional spin, which is all in the same round

hit_percent_per_spin = hit_percent_per_payline * 5
hit_percent_per_spin

25.290755542039722

In [25]:
bonus_game_percent_chance = 100

for reel in df_reels.columns:
    a = np.count_nonzero(df_reels[reel] == 'S')/len(df_reels[reel]) * 3
    if a != 0:
        bonus_game_percent_chance = bonus_game_percent_chance * a

bonus_game_percent_chance

7.871720116618075

In [26]:
any_hit_percent_chance = hit_percent_per_spin + bonus_game_percent_chance
any_hit_percent_chance

33.1624756586578

### 3) Calculating RTP

RTP: % of prizes returned to the player depending on funds deposited during the game initially

```
(money deposited) * sum(every combo probability * respective combo payout multiplier) / (money deposited) * 1
```

```
sum(every combo probability * respective combo payout multiplier)
```

In [27]:
df_combo_probs

,3,4,5
A,0.002539,0.000705,0.000132
K,0.003030,0.001426,0.000439
Q,0.011283,0.003761,0.001645
J,0.015279,0.007052,0.003291


In [28]:
df_dict['Pay Table']

,Unnamed: 0,Coins,Unnamed: 2,Unnamed: 3,Unnamed: 4,payline 1
0,Symbol,x3,x4,x5,NaN,NaN
1,A,5.0,50.0,1000.0,NaN,NaN
2,K,5.0,25.0,250.0,NaN,NaN
3,Q,3.0,10.0,100.0,NaN,NaN
4,J,3.0,10.0,50.0,NaN,payline 2
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,x3,NaN,NaN,NaN,NaN
7,S,3.0,Free spins,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,payline 3


In [29]:
df_multipliers = df_dict['Pay Table'][:5][df_dict['Pay Table'].columns[:4]]
df_multipliers

,Unnamed: 0,Coins,Unnamed: 2,Unnamed: 3
0,Symbol,x3,x4,x5
1,A,5.0,50.0,1000.0
2,K,5.0,25.0,250.0
3,Q,3.0,10.0,100.0
4,J,3.0,10.0,50.0


In [30]:
df_multipliers['Unnamed: 0'].values

array(['Symbol', 'A', 'K', 'Q', 'J'], dtype=object)

In [31]:
df_multipliers.index = df_multipliers['Unnamed: 0'].values
df_multipliers

,Unnamed: 0,Coins,Unnamed: 2,Unnamed: 3
Symbol,Symbol,x3,x4,x5
A,A,5.0,50.0,1000.0
K,K,5.0,25.0,250.0
Q,Q,3.0,10.0,100.0
J,J,3.0,10.0,50.0


In [32]:
df_multipliers.drop('Unnamed: 0', axis=1, inplace=True)
df_multipliers

,Coins,Unnamed: 2,Unnamed: 3
Symbol,x3,x4,x5
A,5.0,50.0,1000.0
K,5.0,25.0,250.0
Q,3.0,10.0,100.0
J,3.0,10.0,50.0


In [33]:
df_multipliers.columns = df_combo_probs.columns
df_multipliers

,3,4,5
Symbol,x3,x4,x5
A,5.0,50.0,1000.0
K,5.0,25.0,250.0
Q,3.0,10.0,100.0
J,3.0,10.0,50.0


In [34]:
df_multipliers.drop('Symbol', inplace=True)
df_multipliers

,3,4,5
A,5.0,50.0,1000.0
K,5.0,25.0,250.0
Q,3.0,10.0,100.0
J,3.0,10.0,50.0


In [35]:
df_combo_probs

,3,4,5
A,0.002539,0.000705,0.000132
K,0.003030,0.001426,0.000439
Q,0.011283,0.003761,0.001645
J,0.015279,0.007052,0.003291


In [36]:
df_payouts = df_multipliers * df_combo_probs
df_payouts

,3,4,5
A,0.012693,0.035259,0.13222
K,0.015151,0.03565,0.109694
Q,0.033848,0.037609,0.16454
J,0.045836,0.070517,0.16454


In [37]:
rtp_percent = df_payouts.sum().sum() * (100 + bonus_game_percent_chance)
rtp_percent

92.50642862839054

### 4) Compiling and recording answers

In [38]:
answers = {
    'Stat name' : ['Hit % per payline', 'Hit % per spin', 'Bonus game %',
                   'Any hit %', 'RTP %'],
    'Value' : [hit_percent_per_payline, hit_percent_per_spin, bonus_game_percent_chance,
               any_hit_percent_chance, rtp_percent]
}

In [39]:
answers = {
    'Hit % per payline' : [hit_percent_per_payline],
    'Hit % per spin' : [hit_percent_per_spin],
    'Bonus game %' : [bonus_game_percent_chance],
    'Any hit %' : [any_hit_percent_chance],
    'RTP %' : [rtp_percent]
}

In [40]:
answers

{'Hit % per payline': [5.0581511084079445],
 'Hit % per spin': [25.290755542039722],
 'Bonus game %': [7.871720116618075],
 'Any hit %': [33.1624756586578],
 'RTP %': [92.50642862839054]}

In [41]:
df_answers = pd.DataFrame(answers)
df_answers

,Hit % per payline,Hit % per spin,Bonus game %,Any hit %,RTP %
0,5.058151,25.290756,7.87172,33.162476,92.506429


In [42]:
excel_file_path = 'output_data.xlsx'

df_answers.to_excel(excel_file_path, index=False)